In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
dataSet_1= 'satimage_'
filePath_1 = '/content/drive/MyDrive/Ms.C. in CSE/October 2022/CSE 6705(Meta Heuristics)/Project/satimage_csv_.csv' 
data_1 = pd.read_csv(filePath_1)
data_1 = data_1.drop(['Unnamed: 0'], axis=1)
data_1 

,Aattr,Battr,Cattr,Dattr,Eattr,Fattr,A1attr,B2attr,C3attr,D4attr,...,D22attr,E23attr,F24attr,A25attr,B26attr,C27attr,D28attr,E29attr,F30attr,Class
0,0.117596,1.241362,1.184036,0.815302,-0.158561,1.256483,1.193546,0.818486,-0.141965,0.879481,...,0.807707,-0.069968,1.219160,1.250463,0.597678,-0.054291,1.233342,1.262255,0.603258,1
1,-1.205362,-1.249654,-0.077532,0.444886,-0.895959,-0.447579,-0.786760,-0.554203,-0.364672,0.092157,...,-0.192752,-0.736996,-0.969292,-0.844805,-0.400030,-0.725852,-0.344432,-0.594534,-0.183967,1
2,0.779075,0.148811,0.042617,-0.243030,0.800057,0.164136,0.053370,-0.448612,0.154978,-0.345245,...,-0.877277,0.671174,-0.006373,-0.425752,-0.662584,0.691889,0.356801,-0.175259,-0.236449,1
3,1.146564,0.585831,0.342991,0.021553,0.947536,0.601074,0.353416,0.026550,1.788164,1.010702,...,0.281150,1.412317,1.044084,0.532085,0.282612,1.438068,1.058033,0.842981,0.130923,1
4,-0.764376,-1.162250,-0.137607,0.180303,-0.969698,-1.146681,-0.126658,0.184937,-0.735851,-1.132569,...,-0.192752,-0.885225,-1.231906,-0.784941,-0.347519,-0.875088,-1.220973,-0.774223,-0.551339,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6425,1.514052,1.110256,1.063887,0.497802,1.242496,0.906932,0.773481,0.343324,1.268514,0.923221,...,0.597084,0.671174,1.394236,0.891274,0.755211,0.691889,1.233342,1.202359,0.603258,1
6426,-0.102897,-0.375613,0.523215,0.709468,0.136399,0.164136,0.533444,0.290528,0.154978,0.354598,...,0.333806,-0.440539,0.168703,0.292626,0.335123,0.094945,0.751244,0.303913,0.130923,0
6427,1.661048,1.241362,0.883663,0.391969,1.316235,0.863238,0.353416,0.026550,0.748864,0.223378,...,-0.192752,0.819403,0.650162,-0.306022,-0.557562,0.542653,0.093838,-0.295052,-0.761265,0
6428,0.117596,-0.200805,-0.858503,-0.825112,-0.453520,-0.360191,-1.086807,-0.818182,-0.141965,-0.170284,...,0.017871,-0.144082,0.037396,-0.365887,-0.557562,0.169563,-0.081470,-0.534637,-0.551339,0


In [3]:
import random

In [4]:
# Define the size of the population and the number of generations
POPULATION_SIZE = 10
NUM_GENERATIONS = 50


In [5]:
def get_common_value(data):
  X=data.values.tolist()
  N=len(data)
  return data,X,N

In [6]:
#determine the index of class 0 and class 1 samples
def classLabelFinding(data):
  class_0_index=[]
  class_1_index=[]
  X=data.values.tolist()
  for index in range(len(X)):
    if(X[index][36] ==1.0):
      class_1_index.append(index)
    else:
      class_0_index.append(index)
  minor_class_sample=len(class_0_index)
  major_class_sample=len(class_1_index)

  return class_0_index, class_1_index

In [7]:
import math
# Define the triplet loss function
def triplet_loss(anchor, positive, negative,data):

    # Calculate the Euclidean distance between the anchor and positive samples
    X=data.values.tolist()
    pos_dist = sum([math.dist(X[a][:36],X[p][:36]) for a, p in zip(anchor, positive)])
    # Calculate the Euclidean distance between the anchor and negative samples
    neg_dist = sum([math.dist(X[a][:36],X[n][:36]) for a, n in zip(anchor, negative)])
    # Calculate the triplet loss
    loss = max(pos_dist - neg_dist + 1, 0)
    return loss

   

In [8]:
# Define the fitness function
def fitness(individual,data):
    # Split the list into anchor, positive and negative samples
    N=len(data)
    anchor = individual[:N]
    positive = individual[N:2*N]
    negative = individual[2*N:]
    # Calculate the triplet loss for the triplet set
    loss = triplet_loss(anchor, positive, negative,data)
    return -loss  # We maximize the negative of the triplet loss

In [9]:
# Define the initialization function
def initialize_population(data):
    # Create a list of random individuals
    population = []
    N = len(data)
    X=data.values.tolist()

    class_0_index,class_1_index=classLabelFinding(data)
    
    for pop_no in range(POPULATION_SIZE):
      individual= [0 for j in range(3*N)]
      for i in range(len(X)):
        individual[i]=i
        if(X[i][36] ==1.0):
          individual[N+i]=random.choice(class_1_index)
          individual[2*N+i]=random.choice(class_0_index)
        else:
          individual[N+i]=random.choice(class_0_index)
          individual[2*N+i]=random.choice(class_1_index)
        population.append(individual)
      return population

In [10]:
# Define the selection function
def selection(population,data):
    # Select two individuals randomly from the population
    i1, i2 = random.sample(range(POPULATION_SIZE), 2)
    # Return the fitter individual
    if fitness(population[i1],data) > fitness(population[i2],data):
       
        return population[i1]
    else:
        return population[i2]

In [11]:
# Define the crossover function
def crossover(individual1, individual2):
    # Perform uniform crossover
    child = [i1 if random.random() < 0.5 else i2 for i1, i2 in zip(individual1, individual2)]
    return child

In [12]:
# Define the mutation function
def mutation(individual,data):
    # Flip a random bit with a small probability
    class_0_index,class_1_index=classLabelFinding(data)
    N=len(data)
    X=data.values.tolist()
    for i in range(len(individual)):
        if random.random() < 0.01:
         # print("1")
          if(i>=N and i<2*N):
            index=i-N
           # print("2")
            if(X[index][36] ==1.0):
              individual[N+index]=random.choice(class_1_index)
             # print("3")
            else:
              individual[N+index]=random.choice(class_0_index)
              #print("4")
          elif(i>=2*N):
            index=i-(2*N)
           # print("5")
            if(X[index][36] ==1.0):
              #print("6")
              individual[2*N+index]=random.choice(class_0_index)
            else:
              #print("7")
              individual[2*N+index]=random.choice(class_1_index)
          else:  
            pass
    return individual

   

In [13]:
def get_triplet_list(individual,data):
  N=len(data)
  X=data.values.tolist()
  triplet_list=[]
  for i in range(N):
    #print("i:",i)
    sample_triplet=[]
    a_index=individual[i]
    #print(a_index)
    p_index=individual[N+i]
    #print(p_index)
    n_index=individual[2*N+i]
    #print(n_index)
    anchor=X[a_index][0:36]
   # print("Anchor")
    #print(anchor)
    positive=X[p_index][0:36]
    #print("Positive")
    #print(positive)
    negative=X[n_index][0:36]
    #print("Negative")
    #print(negative)
    sample_triplet.append(anchor)
    sample_triplet.append(positive)
    sample_triplet.append(negative)
   # print(sample_triplet)
    triplet_list.append(sample_triplet)
  
  return triplet_list

In [14]:
# Define the genetic algorithm function
def genetic_algorithm(data):

    
    # Initialize the population
    population = initialize_population(data)
    fitted_population=[]
    # Iterate over the generations
    for generation in range(NUM_GENERATIONS):
        # Create a new population
        new_population = []
        # Generate offspring
        for i in range(POPULATION_SIZE):
            # Select parents
            parent1 = selection(population,data)
            parent2 = selection(population,data)
            # Crossover parents
            child = crossover(parent1, parent2)
            # Mutate child
            child = mutation(child,data)
            # Add child to new population
            new_population.append(child)
        # Replace the old population with the new population
        population = new_population
        fitness_value=[]
        for i in range(len(population)):
          fit_value=fitness(population[i],data)
          fitness_value.append(fit_value)
        max_fit=max(fitness_value)
        for i in range(len(population)):
          if(fitness_value[i]==max_fit):
            fittest_individual=population[i]
            break
            

      
        fitted_population.append(fittest_individual)
        # Find the fittest individual in the population
        #fittest_individual = max(population, key=fitness(x1,data))
        # Print the fittest individual and its fitness
        #print("Generation ", generation+1, ": ", fittest_individual, "\n Fitness:", -fitness(fittest_individual,data))
        #print("Generation ", generation+1, ": ", fittest_individual, "\n Fitness:", -fitness(fittest_individual,data),"\n Triplet_list", get_triplet_list(fittest_individual,data))
        
    return fitted_population

# **Train test spliting**

In [15]:
import numpy as np
from sklearn.model_selection import train_test_split

indices_1 = np.arange(len(data_1))

y_1 = data_1['Class']
X_1 = data_1
X_1 = X_1.drop(['Class'], axis=1)
x1, x2, y1, y2, idx1, idx2 = train_test_split(X_1, y_1, indices_1, test_size=0.2)

#constructing pd tables for train and test data
x1["Class"]=y1
x2["Class"]=y2
 
#making list of train and test data
# train_data=x1.values.tolist()
# test_data=x2.values.tolist()
train_data_1=x1
test_data_1=x2

# **Final output**

In [16]:
fitted_train_triplet_1=genetic_algorithm(train_data_1)
fitted_test_triplet_1=genetic_algorithm(test_data_1)

In [17]:
from pprint import pprint
import numpy as np

In [18]:

x=get_triplet_list(fitted_train_triplet_1[49],train_data_1)

y=get_triplet_list(fitted_test_triplet_1[49],test_data_1)

In [19]:
train_triplet_array=np.array(x)
test_triplet_array=np.array(y)
print(train_triplet_array)
print(test_triplet_array)
print(train_triplet_array.shape)
print(test_triplet_array.shape)

[[[ 0.117596  1.241362  1.48441  ...  1.40865   1.262255  0.813184]
  [ 1.367057  0.848043  0.282916 ...  1.145687  1.082566  0.340849]
  [-0.176394 -0.463017 -0.798429 ...  0.093838 -0.055466 -0.183967]]

 [[ 0.117596  1.19766   1.123962 ...  1.277169  1.202359  0.813184]
  [ 0.117596 -0.200805 -0.618205 ...  0.050011 -0.774223 -0.393894]
  [ 1.293559  0.848043  0.342991 ...  0.838898  0.18412   0.025959]]

 [[-0.249892 -0.200805 -0.918578 ... -0.51974  -1.013809 -0.91871 ]
  [-0.396887  0.629533  0.703439 ...  0.006184  0.303913  0.025959]
  [-0.396887 -0.463017 -0.798429 ...  0.707417  0.064327 -0.183967]]

 ...

 [[-0.249892  0.148811  0.282916 ...  0.400628  0.363809  0.235886]
  [-1.205362 -1.424462 -1.939848 ... -1.177146 -1.792462 -1.2336  ]
  [ 0.338089  0.323619  0.042617 ...  0.225319 -0.354948 -0.341412]]

 [[-1.205362 -1.424462 -1.939848 ... -1.483936 -1.373187 -1.2336  ]
  [-0.396887 -0.637825 -1.519325 ... -0.344432 -1.313291 -1.286082]
  [ 0.338089  0.367321 -0.618205 .

In [20]:
np.save('satimage_train.npy',train_triplet_array)
np.save('satimage_test.npy',test_triplet_array)